In [105]:
# Dependencies
import pandas as pd
import numpy as np
from collections import defaultdict
import re

# Make sure you can see all output
pd.options.display.max_rows = 4000

In [106]:
# Store filepath in a variable
dfAll = pd.read_csv("Resources/all_data.csv", encoding="ISO-8859-1")
dfAll.head()

,ï»¿RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Notes_END,Notes_DURATION,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,DS,rPPA,15,WAB Naming,1,Baseline,NaN,NaN,NaN,NaN,...,1900.200674,1900.200674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DS,rPPA,15,WAB Naming,1,Baseline,NaN,NaN,NaN,NaN,...,1900.496319,0.295646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1900.496319,0.295646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1900.496319,0.295646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1900.496319,0.295646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
# Drop unnamed columns
dfAll = dfAll.loc[:, ~dfAll.columns.str.contains('^Unnamed')]

# Remove spaces from column names
dfAll.columns = dfAll.columns.str.replace(' ', '')

In [108]:
dfAll.columns.to_list()

['ï»¿RA',
 'Project',
 'PID',
 'Test',
 'Arm',
 'Week',
 'Target',
 'Production',
 'PRODUCTION_START',
 'PRODUCTION_END',
 'PRODUCTION_DURATION',
 'Phoneme',
 'PHONEME_START',
 'PHONEME_END',
 'PHONEME_DURATION',
 'PVOTVLCD',
 'PVOTVLCD_START',
 'PVOTVLCD_END',
 'PVOTVLCD_DURATION',
 'HesPauseAmb',
 'HesPauseAmb_START',
 'HesPauseAmb_END',
 'HesPauseAmb_DURATION',
 'NOTES',
 'Notes_START',
 'Notes_END',
 'Notes_DURATION']

In [109]:
# Create a subset of data based on data you are interested in working with
df = dfAll[
    ['ï»¿RA',
    'Project',
    'PID',
    'Test',
    'Arm',
    'Week',
    'Target',
    'Production',
    'PRODUCTION_START',
    'PRODUCTION_END',
    'PRODUCTION_DURATION',
    'NOTES']
       ].copy()

# Rename the columns to something easier to work with
df.rename(
    columns={
       'ï»¿RA':'RA'
       }, inplace=True)

# Drop duplicates and NaNs
df = df.dropna(subset=['Target','Production']).drop_duplicates()

# Check data
df.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,PRODUCTION_DURATION,NOTES
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word
8,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,NaN
25,DS,rPPA,15,WAB Naming,1,Baseline,knife,SH,1928.584784,1928.765644,0.180860,Uncertain whether attempt or hesitation
37,DS,rPPA,15,WAB Naming,1,Baseline,knife,SH,1939.241356,1939.408798,0.167443,Unclear whether attempt or hesitation
45,DS,rPPA,15,WAB Naming,1,Baseline,knife,N,1946.663867,1948.174319,1.510452,"Rather quiet utterance of ""n"""


In [110]:
# Check how many participants we have
df['PID'].nunique()

9

In [111]:
#Check that target words are correctly written
df['Target'].unique()

array(['book', 'ball', 'knife', 'cup', 'safety pin', 'hammer',
       'toothbrush', 'eraser', 'lock', 'pencil', 'screwdriver', 'key',
       'paper clip', 'watch', 'comb', 'rubber band', 'spoon', 'tape',
       'fork', 'matches', 'toothpaste', 'ball ', 'matches ', 'pen',
       'rubber band '], dtype=object)

In [112]:
#Replace incorrect target words
df['Target'] = (
    df['Target']
    .replace(
        {
        'pen':'pencil', 
        'paper':'paper clip',
        'ball ':'ball',
        'toothpaste':'toothbrush',
        'matches ':'matches',
        'rubber band ':'rubber band'
        }
    )
)

#Get rid of the one that couldn't be matched to a specific target
df = df.loc[df["Target"] != 'S UH Q IH']

#Check to see if they all look good now
df['Target'].unique()

array(['book', 'ball', 'knife', 'cup', 'safety pin', 'hammer',
       'toothbrush', 'eraser', 'lock', 'pencil', 'screwdriver', 'key',
       'paper clip', 'watch', 'comb', 'rubber band', 'spoon', 'tape',
       'fork', 'matches'], dtype=object)

In [113]:
sorted(df['Production'].unique().tolist())

[' F OW R K',
 ' N AY F',
 ' T UY',
 'AA . P AH N',
 'AA R . Y UW',
 'AE . CH IH Z',
 'AE M . AXR',
 'AX . R EY . S AXR',
 'AX . R EY . S ER',
 'AX R EY . S AXR',
 'AX R EY . S AXR ',
 'AXR . H AE . M AXR',
 'AY',
 'AY F',
 'B',
 'B  UH K',
 'B AA',
 'B AA ',
 'B AA . B IY . P IH N',
 'B AA L',
 'B AH',
 'B AH S',
 'B AO L',
 'B AX',
 'B AXR . B AE N D',
 'B IH T',
 'B IY',
 'B IY CH',
 'B IY S',
 'B OW T',
 'B R AH SH',
 'B R UH . CH AXR . T IY TH',
 'B R UH K',
 'B R UH SH',
 'B R UH SH . T IY TH',
 'B UH K',
 'B UH K . B R UH K',
 'B UH SH',
 'B UW T S',
 'CH',
 'CH AA R . JH',
 'D AA G',
 'D AO',
 'D AO G',
 'D EY',
 'D IY',
 'D IY . OW . Q AH R',
 'D R AY',
 'D R AY . V AXR',
 'EH',
 'EH . R EY . S AXR',
 'EH . R EY . S ER',
 'EH L',
 'EH S',
 'EY',
 'F',
 'F AO R',
 'F AO R . W EY ',
 'F AO R K',
 'F AO R K ',
 'F AX',
 'F AY . AXR',
 'F OA R K',
 'F OW R K',
 'F R AA G',
 'F UH T',
 'G EH . DX IH N',
 'G UH . D AA . K AA R L',
 'H',
 'H AA R',
 'H AE',
 'H AE . M AX R',
 'H AE .

In [114]:
#Replace problematic transcriptions
df['Production'] = (
    df['Production']
    .replace(
        {
        ' F OW R K':'F OW R K', 
        ' N AY F':'N AY F',
        ' T UY':'T UY',
        'B UWH K':'B UH K',
        'B  UH K':'B UH K',
        'F AO R . W EY ':'F AO R . W EY',
        'F AO R K ':'F AO R K',
        'M AE . CH IH S':'M AE . CH IH Z',
        'M AE . CH IX':'M AE . CH IH',
        'M AE CH . IX S': 'M AE CH . IH Z',
        'P EY . P ER . K L IH ':'P EY . P ER . K L IH',
        'W AA CH ':'W AA CH',
        'N IX ':'N IH',
        'P EY P AXR . K LIH P':'P EY P AXR . K L IH P',
        'SEYF . SEYF . T IY':'S EY F . S EY F . T IY',
        'SEYF . T IY':'S EY F . T IY',
        'P EY . P AXRR':'P EY . P AXR',
        'P EH N . S AXL':'P EH N . S AX L',
        'SK R UW . D R AY V AXR':'S K R UW . D R AY V AXR',
        'F OA R K':'F AO R K',
        'F OW R K':'F AO R K',
        'T UX':'T UW',
        'T UX SH . B OW N':'T UW SH . B OW N',
        'T UY': 'T UW',
        'T UY TH . P IY S':'T UW TH . P IY S',
        'S K R U . JH AXR':'S K R UW . JH AXR',
        'R UH . B ER . B A N D':'R UH . B ER . B AE N D'
        }
    )
)

In [115]:
#Check what each phoneme is being registered as
results = (
    # trans is a series, so use string accessor to split value strings
    df["Production"].str.split(" ")
    # turn each item in split string into own row maintaining index value
    .explode()
)

results.unique()

array(['B', 'UH', 'K', 'AO', 'L', 'SH', 'N', 'AY', 'F', 'AH', 'P', 'S',
       '.', 'T', 'IY', 'H', 'AE', 'M', 'ER', 'UW', 'OW', 'TH', 'UY', 'R',
       'AXR', 'AA', 'EH', 'AX', 'DX', 'CH', 'ZH', 'IH', 'EY', 'W', 'Q',
       'EI', '', 'JH', 'D', 'NG', 'G', 'Z', 'V', 'Y'], dtype=object)

In [165]:
#See all rows for alone 'U' items to get string environment for replacement
p = '\sU\s'
df[df['Production'].str.contains(p)]

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,PRODUCTION_DURATION,NOTES


In [117]:
# Replace extra substrings 
df['Production'] = (
    df['Production']
    .replace(
            {
           # '\.|\..':'',
            'UY':'UW',
            'EI':'EY'
            },
        regex=True)
    .str.strip()
)


In [118]:
# Replace extra spaces 
df['Production'] = (
    df['Production']
    .replace(
            {
            '\s\s|\s\s\s':' '
            },
        regex=True)
    .str.strip()
)


In [119]:
#Recheck what each phoneme is being registered as
results = (
    # trans is a series, so use string accessor to split value strings
    df["Production"].str.split(" ")
    # turn each item in split string into own row maintaining index value
    .explode()
)

results.unique()

array(['B', 'UH', 'K', 'AO', 'L', 'SH', 'N', 'AY', 'F', 'AH', 'P', 'S',
       '.', 'T', 'IY', 'H', 'AE', 'M', 'ER', 'UW', 'OW', 'TH', 'R', 'AXR',
       'AA', 'EH', 'AX', 'DX', 'CH', 'ZH', 'IH', 'EY', 'W', 'Q', 'JH',
       'D', 'NG', 'G', 'Z', 'V', 'Y'], dtype=object)

In [120]:
#Some of the words still have some extra spaces (determined by error message when trying to convert the productions to IPA), so going to delete all spaces and then add them back in

df["Production"] = (
    # Production is a series, so use string accessor to split value strings
    df["Production"].str.split(" ")
    # turn each item in split string into own row maintaining index value
    .explode()
    # Remove blank
    .dropna()
    # group them by the original index
    .groupby(level=0)
    # aggregate them with a space
    .agg(lambda s: " ".join(s))
    
)

df["Production"].unique()

array(['B UH K', 'B AO L', 'SH', 'N', 'N AY F', 'K AH P', 'S',
       'S AY F . T IY', 'H AE . M ER', 'T UW SH . B OW N', 'T UW',
       'B UH SH', 'T UH', 'T IY', 'T UW TH . P IY S', 'T P IY', 'T IY S',
       'P IY', 'P', 'IY', 'R AH B', 'R AH . B AXR N', 'R AH . B AXR',
       'L AA . K AXR', 'T EH M', 'T', 'P EH N . S AX L', 'S T R UW',
       'S T R', 'S T R UW . DX OW', 'S K R UW', 'S K R UW CH',
       'S K R UW S', 'K', 'ZH', 'K IY', 'T S', 'S IH M', 'P EY . P AXR',
       'W AA CH', 'OW', 'R AH . B AXR . L', 'R AH . B AXR . N AX Q',
       'R AH . B AXR . N', 'R AH', 'K AH', 'Q', 'S P UW', 'S P UH SH',
       'S P', 'S B AH', 'S UW P', 'T EY', 'SH AO R', 'B AX', 'AY F', 'AY',
       'K AO', 'K AH F', 'K AO Q', 'K AO F', 'K AH Q', 'S T EY Q',
       'S T EY', 'S T AY', 'H', 'H AE', 'H AE . M AXR', 'T IH TH',
       'P EY S', 'B AA', 'P AH', 'B', 'B AH', 'L', 'L AA K',
       'P EH N . CH R', 'P EH N . S IH', 'P EH N', 'P EH', 'P IH N',
       'K OW', 'K AX', 'S T IH F', 'S P UW

In [121]:
df.reset_index(inplace=True,drop=True)
df.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,PRODUCTION_DURATION,NOTES
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word
1,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,NaN
2,DS,rPPA,15,WAB Naming,1,Baseline,knife,SH,1928.584784,1928.765644,0.180860,Uncertain whether attempt or hesitation
3,DS,rPPA,15,WAB Naming,1,Baseline,knife,SH,1939.241356,1939.408798,0.167443,Unclear whether attempt or hesitation
4,DS,rPPA,15,WAB Naming,1,Baseline,knife,N,1946.663867,1948.174319,1.510452,"Rather quiet utterance of ""n"""


In [122]:
Phoneme = (
    # Production is a series, so use string accessor to split value strings
    df["Production"].str.split(" ")
    # turn each item in split string into own row maintaining index value
    .explode()
    )

In [123]:
# Merge phonemes with data
df2 = pd.merge(df, Phoneme, left_index=True, right_index=True)

#Reset indec
df2.reset_index(inplace=True,drop=True)

df2 = df2.rename(columns={'Production_x':'Production','Production_y':'Phoneme'})

In [124]:
df2.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,PRODUCTION_DURATION,NOTES,Phoneme
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,B
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,UH
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,K
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,NaN,B
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,NaN,AO


In [125]:
#Create function to assign a unique ID to words
word_id = defaultdict(lambda: len(word_id))
df2['Word_ID'] = [word_id[i]+1 for i in df2['Target']]

df2.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,PRODUCTION_DURATION,NOTES,Phoneme,Word_ID
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,B,1
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,UH,1
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,K,1
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,NaN,B,2
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,NaN,AO,2


In [126]:
#Assign a numeric ID to each session

df2['Session_ID'] = df2["Arm"].astype(str) + df2["Week"].astype(str)

df2['Session_ID'] = (
    df2['Session_ID']
    .replace(
        {
        '1Baseline':'0', 
        '112':'1',
        '212':'2'
        }
    )
)
df2['Session_ID'].unique()

array(['0', '1', '2'], dtype=object)

In [127]:
#Assign production number to label when each production attempt was made for each word
df2['Prod_Word_N'] = df2.groupby(['PID','Session_ID'])['PRODUCTION_START'].rank('dense').astype(int)

# Reset index and drop unneeded one
df2.reset_index(inplace=True,drop=True)
df2.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,PRODUCTION_DURATION,NOTES,Phoneme,Word_ID,Session_ID,Prod_Word_N
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,B,1,0,1
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,UH,1,0,1
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,K,1,0,1
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,NaN,B,2,0,2
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,NaN,AO,2,0,2


In [128]:
#Add a new column for repeated index
df2.reset_index(inplace=True)

In [129]:
#Assign production number to label when each production attempt was made for each phoneme
df2['Prod_Phon_N'] = df2.groupby(['PID','Session_ID','PRODUCTION_START'])['index'].rank('dense').astype(int)

#drop unneeded index column
df2 = df2.drop(columns={'index'})

#check
df2.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,PRODUCTION_DURATION,NOTES,Phoneme,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,B,1,0,1,1
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,UH,1,0,1,2
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,0.295646,Article (É) before word,K,1,0,1,3
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,NaN,B,2,0,2,1
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,0.397365,NaN,AO,2,0,2,2


In [130]:
#Create a code to tie each participant to their session
df2['PIDSESS_Code'] = (df2["Project"]
                       .astype(str) + 
                       '-' + 
                       df2["PID"].astype(str).str.zfill(2) + 
                       '_' + 
                       df2["Session_ID"].astype(str)
                       )

df2['PIDSESS_Code'].unique()

array(['rPPA-15_0', 'rPPA-15_1', 'rPPA-15_2', 'rPPA-01_0', 'rPPA-01_1',
       'rPPA-01_2', 'rPPA-08_0', 'rPPA-08_1', 'rPPA-08_2', 'rPPA-12_0',
       'rPPA-12_2', 'dPPA-09_0', 'dPPA-09_1', 'dPPA-09_2', 'rPPA-12_1',
       'rPPA-05_0', 'rPPA-05_1', 'rPPA-05_2', 'rPPA-13_0', 'rPPA-13_1',
       'rPPA-13_2', 'rPPA-16_0', 'rPPA-16_1', 'rPPA-16_2', 'rPPA-07_0',
       'rPPA-07_1', 'rPPA-07_2'], dtype=object)

In [131]:
#Create a code to tie each word and phoneme together based on phoneme position
df2['WordPhon_Code'] = df2["Word_ID"].astype(str) + str('_') + df2["Prod_Phon_N"].astype(str)

#Create a separate code to tie each participant to their word-phoneme production pairs
df2['Phon_Sess_Code'] = (
    df2["PID"].astype(str) + 
    str('_') + 
    df2["Session_ID"].astype(str) + 
    str('_') + 
    df2["Prod_Word_N"].astype(str) +
    str('_') + 
    df2["Prod_Phon_N"].astype(str)
    .astype(str)
)

#Create a separate code to tie each participant to their word productions
df2['Word_Sess_Code'] = (
    df2["PID"].astype(str) + 
    str('_') + 
    df2["Session_ID"].astype(str) + 
    str('_') + 
    df2["Prod_Word_N"].astype(str)
    .astype(str)
)

df2.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,NOTES,Phoneme,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N,PIDSESS_Code,WordPhon_Code,Phon_Sess_Code,Word_Sess_Code
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,Article (É) before word,B,1,0,1,1,rPPA-15_0,1_1,15_0_1_1,15_0_1
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,Article (É) before word,UH,1,0,1,2,rPPA-15_0,1_2,15_0_1_2,15_0_1
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,Article (É) before word,K,1,0,1,3,rPPA-15_0,1_3,15_0_1_3,15_0_1
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,NaN,B,2,0,2,1,rPPA-15_0,2_1,15_0_2_1,15_0_2
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,NaN,AO,2,0,2,2,rPPA-15_0,2_2,15_0_2_2,15_0_2


In [132]:
#Before we can do anything with the syllables we need to identify the last phoneme produced within each production attempt

last_prod_phon = df2.groupby('Word_Sess_Code')['Prod_Phon_N'].last().to_frame()

last_prod_phon['Prod_Last_Phon'] = 1

last_prod_phon = last_prod_phon.reset_index()

last_prod_phon['Phon_Sess_Code'] = (
    last_prod_phon["Word_Sess_Code"].astype(str) + 
    str('_') + 
    last_prod_phon["Prod_Phon_N"].astype(str)
)

last_prod_phon = last_prod_phon.drop(columns=['Word_Sess_Code','Prod_Phon_N'])

last_prod_phon.head()

,Prod_Last_Phon,Phon_Sess_Code
0,1,12_0_1_3
1,1,12_0_10_7
2,1,12_0_11_6
3,1,12_0_12_17
4,1,12_0_13_2


In [133]:
#Add the identification for the last phoneme produced to the main dataset
df2 = pd.merge(df2, last_prod_phon, on='Phon_Sess_Code', how='left')
df2['Prod_Last_Phon'] = df2['Prod_Last_Phon'].fillna(0).astype('int')
df2.head(5)

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Phoneme,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N,PIDSESS_Code,WordPhon_Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,B,1,0,1,1,rPPA-15_0,1_1,15_0_1_1,15_0_1,0
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,UH,1,0,1,2,rPPA-15_0,1_2,15_0_1_2,15_0_1,0
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,K,1,0,1,3,rPPA-15_0,1_3,15_0_1_3,15_0_1,1
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,B,2,0,2,1,rPPA-15_0,2_1,15_0_2_1,15_0_2,0
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,AO,2,0,2,2,rPPA-15_0,2_2,15_0_2_2,15_0_2,0


In [134]:
#Translate the ARPABET codes to IPA codes 
# NOTE:An error at this step will tell you if any other transcriptions are messed up
dictionary = (
    pd.read_csv("Resources/dict.csv")
    .set_index("Arpabet")
)

df2["Production_IPA"] = (
    # The production column is a series, so use string accessor to split value strings
    df2["Production"].str.split(" ")
    # turn each item in split string into own row maintaining index value
    .explode()
    # perform the lookup in the dictionary of each individual value
    .apply(lambda v: dictionary.loc[v])
    # group them by the original index
    .groupby(level=0)
    # "sum" them, which for string, concatonates them without any spaces
    .sum()
)

df2.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N,PIDSESS_Code,WordPhon_Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon,Production_IPA
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1,0,1,1,rPPA-15_0,1_1,15_0_1_1,15_0_1,0,bʊk
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1,0,1,2,rPPA-15_0,1_2,15_0_1_2,15_0_1,0,bʊk
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1,0,1,3,rPPA-15_0,1_3,15_0_1_3,15_0_1,1,bʊk
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,2,0,2,1,rPPA-15_0,2_1,15_0_2_1,15_0_2,0,bɔl
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,2,0,2,2,rPPA-15_0,2_2,15_0_2_2,15_0_2,0,bɔl


In [135]:
#Import phonetic feature identifies
phon_dist_features = (
    pd.read_csv("Resources/phon_dist_features.csv")
)
phon_dist_features = phon_dist_features.dropna()
phon_dist_features['Phoneme_ID'] = phon_dist_features['Phoneme_ID'].astype('int')

In [136]:
#Create a dictionary for phoneme ID number

phon_ID = phon_dist_features[['IPA_singles','Arpabet','Phoneme_ID']]

phon_ID.rename(
    columns={
       'IPA_singles':'IPA_singles', 
       'Arpabet':'Phoneme'
       }, inplace=True)

phon_ID

/var/folders/b9/y7yy3gjx1vjcj8dmj6zw15kw0000gn/T/ipykernel_18079/3682838066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phon_ID.rename(


,IPA_singles,Phoneme,Phoneme_ID
0,h,H,1
1,r,R,2
2,w,W,3
3,j,Y,4
4,b,B,5
5,ʧ,CH,6
6,d,D,7
7,ð,DH,8
8,ɾ,DX,9
9,f,F,10


In [137]:
#Merge the dictionary to the main dataset on Phoneme, so that corresponding IPA symbols are added as a column
df3 = pd.merge(df2, phon_ID, on='Phoneme', how='left')
df3.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Prod_Word_N,Prod_Phon_N,PIDSESS_Code,WordPhon_Code,Phon_Sess_Code,Word_Sess_Code,Prod_Last_Phon,Production_IPA,IPA_singles,Phoneme_ID
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1,1,rPPA-15_0,1_1,15_0_1_1,15_0_1,0,bʊk,b,5
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1,2,rPPA-15_0,1_2,15_0_1_2,15_0_1,0,bʊk,ʊ,36
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,1,3,rPPA-15_0,1_3,15_0_1_3,15_0_1,1,bʊk,k,13
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,2,1,rPPA-15_0,2_1,15_0_2_1,15_0_2,0,bɔl,b,5
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,2,2,rPPA-15_0,2_2,15_0_2_2,15_0_2,0,bɔl,ɔ,30


In [138]:
#Add in word information file for target items
word_info = (
    pd.read_csv("Resources/word_info.csv")
)

#Drop unneeded columns and rename other columns
word_info = (
    word_info
    .drop(columns=['N_Tot_Words.1','Target','Target_Word_Struc','Target_Syll_Struc'])
    .rename(columns={
        'Arpabet':'Target_Arpabet',
        'N_Tot_Syllables':'Target_N_Tot_Syllables',
        'N_Tot_Target_Characters':'Target_N_Tot_Characters',
        'N_Tot_Words':'Target_N_Tot_Words'
        })
)

word_info.head()

,Target_Arpabet,Word_ID,Target_N_Tot_Words,Target_IPA,Target_N_Tot_Syllables,Target_N_Tot_Characters
0,B UH K,1,1,bʊk,1,3
1,B AO L,2,1,bɔl,1,3
2,N AY F,3,1,nɐf,1,3
3,K AH P,4,1,kʌp,1,3
4,S EY F . T IY . P IH N,5,2,sef.ti.pɪn,3,10


In [139]:
# Merge word information with the main dataset
df4 = pd.merge(df3, word_info, on='Word_ID', how='left')
df4.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Word_Sess_Code,Prod_Last_Phon,Production_IPA,IPA_singles,Phoneme_ID,Target_Arpabet,Target_N_Tot_Words,Target_IPA,Target_N_Tot_Syllables,Target_N_Tot_Characters
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,15_0_1,0,bʊk,b,5,B UH K,1,bʊk,1,3
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,15_0_1,0,bʊk,ʊ,36,B UH K,1,bʊk,1,3
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,15_0_1,1,bʊk,k,13,B UH K,1,bʊk,1,3
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,15_0_2,0,bɔl,b,5,B AO L,1,bɔl,1,3
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,15_0_2,0,bɔl,ɔ,30,B AO L,1,bɔl,1,3


In [140]:
#Load the phonological information for the target items
word_phon_info = (
    pd.read_csv("Resources/word_phon_info.csv")
)

#Drop unneeded columns and rename other columns
word_phon_info = (
    word_phon_info
    .drop(columns=[
        'Word',
        'Word_ID',
        'Production_N'
        ])
    .rename(columns={
        'Phoneme':'Target_Phon_Arpabet',
        'Phoneme_ID':'Target_Phoneme_ID',
        'Word_NumID':'Target_Word_NumID',
        'Con_Cluster':'Target_Con_Cluster',
        'Clust_ID':'Target_Clust_ID',
        'Clus_Type':'Target_Clus_Type',
        'Clust_Phon_Pos':'Target_Clust_Phon_Pos',
        'Clust_Phon_Env':'Target_Clust_Phon_Env',
        'PhonClus_Syll_Env':'Target_Syll_Env',
        'Phoneme_Pos_Word':'Target_Word_Pos',
        'Target_IPA':'Target_Phon_IPA',
        'Code':'WordPhon_Code'
        })
)

#Get target IPA symbols
word_phon_info["Target_Phon_IPA"] = (
    # trans is a series, so use string accessor to split value strings
    word_phon_info["Target_Phon_Arpabet"]
    # perform the lookup in the dictionary of each individual value
    .apply(lambda v: dictionary.loc[v])
)

word_phon_info.head()

,Target_Phon_Arpabet,WordPhon_Code,Target_Phoneme_ID,Target_Syll_Env,Target_Word_Pos,Syllable_NumID,Target_Word_NumID,Target_Con_Cluster,Target_Clust_ID,Target_Clus_Type,Target_Clust_Phon_Pos,Target_Clust_Phon_Env,Target_Phon_IPA
0,B,1_1,5,#_V,#_V,1,1,0,NaN,NaN,NaN,NaN,b
1,UH,1_2,36,C_C,C_C,1,1,0,NaN,NaN,NaN,NaN,ʊ
2,K,1_3,13,V_#,V_#,1,1,0,NaN,NaN,NaN,NaN,k
3,B,2_1,5,#_V,#_V,1,1,0,NaN,NaN,NaN,NaN,b
4,AO,2_2,30,C_C,C_C,1,1,0,NaN,NaN,NaN,NaN,ɔ


In [141]:
# Merge target phonological information with the main dataset
df5 = pd.merge(df4, word_phon_info, on='WordPhon_Code', how='left')
df5.head()

,RA,Project,PID,Test,Arm,Week,Target,Production,PRODUCTION_START,PRODUCTION_END,...,Target_Syll_Env,Target_Word_Pos,Syllable_NumID,Target_Word_NumID,Target_Con_Cluster,Target_Clust_ID,Target_Clus_Type,Target_Clust_Phon_Pos,Target_Clust_Phon_Env,Target_Phon_IPA
0,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,#_V,#_V,1.0,1.0,0.0,NaN,NaN,NaN,NaN,b
1,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,C_C,C_C,1.0,1.0,0.0,NaN,NaN,NaN,NaN,ʊ
2,DS,rPPA,15,WAB Naming,1,Baseline,book,B UH K,1900.200674,1900.496319,...,V_#,V_#,1.0,1.0,0.0,NaN,NaN,NaN,NaN,k
3,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,#_V,#_V,1.0,1.0,0.0,NaN,NaN,NaN,NaN,b
4,DS,rPPA,15,WAB Naming,1,Baseline,ball,B AO L,1905.496713,1905.894079,...,C_C,C_C,1.0,1.0,0.0,NaN,NaN,NaN,NaN,ɔ


In [142]:
#Clean up main dataset's columns and rename things
df6 = (
    df5
    .drop(columns=[
        'PRODUCTION_START',
        'PRODUCTION_END'
        ])
    .rename(columns={
        'Phoneme':'Prod_Arpabet',
        'IPA_singles':'Prod_Phon_IPA',
        'Production_IPA':'Prod_Word_IPA',
        'Phoneme_ID':'Prod_Phoneme_ID',
        'Target_IPA':'Target_Word_IPA',
        'PRODUCTION_DURATION':'Prod_Word_Dur'
        })
)

#Fill in NAs with integer or string values, depending on column
df6['Target_Phoneme_ID'] = df6['Target_Phoneme_ID'].fillna(44).astype('int')
df6['Target_Syll_Env'] = df6['Target_Syll_Env'].fillna('addition').astype('str')
df6['Target_Word_Pos'] = df6['Target_Word_Pos'].fillna('addition').astype('str')

# Check
df6[27:33][['Prod_Word_IPA','Target_Word_IPA','Prod_Phon_IPA', 'Target_Phon_IPA']]


,Prod_Word_IPA,Target_Word_IPA,Prod_Phon_IPA,Target_Phon_IPA
27,sɐf.ti,sef.ti.pɪn,t,t
28,sɐf.ti,sef.ti.pɪn,i,i
29,hæ.mɝ,hæ.mɚ,h,h
30,hæ.mɝ,hæ.mɚ,æ,æ
31,hæ.mɝ,hæ.mɚ,.,.
32,hæ.mɝ,hæ.mɚ,m,m


In [143]:
#Assign target phonetic features
target_phon_dist_features = (
    phon_dist_features
    .drop(columns=[
        'IPA_singles',
        'Arpabet'
        ])
    .add_prefix("Target_")
)
target_phon_dist_features.head()

,Target_Phoneme_ID,Target_syllabic,Target_consonantal,Target_sonorant,Target_continuant,Target_delayed release,Target_approximant,Target_tap,Target_nasal,Target_voice,...,Target_velar,Target_alveolar,Target_post-alveolar,Target_dental,Target_palatal,Target_glottal,Target_stop,Target_fricative,Target_affricate,Target_glide
0,1,-1,-1,-1,1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,1,-1,1,-1,-1
1,2,-1,-1,1,1,0,1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,-1
2,3,-1,-1,1,1,0,1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
3,4,-1,-1,1,1,0,1,-1,-1,1,...,-1,-1,-1,-1,1,-1,-1,-1,-1,1
4,5,-1,1,-1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1


In [144]:
#Assign production phonetic features
prod_phon_dist_features = (
    phon_dist_features
    .drop(columns=[
        'IPA_singles',
        'Arpabet'
        ])
    .add_prefix("Prod_")
)
prod_phon_dist_features.head()

,Prod_Phoneme_ID,Prod_syllabic,Prod_consonantal,Prod_sonorant,Prod_continuant,Prod_delayed release,Prod_approximant,Prod_tap,Prod_nasal,Prod_voice,...,Prod_velar,Prod_alveolar,Prod_post-alveolar,Prod_dental,Prod_palatal,Prod_glottal,Prod_stop,Prod_fricative,Prod_affricate,Prod_glide
0,1,-1,-1,-1,1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,1,-1,1,-1,-1
1,2,-1,-1,1,1,0,1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,-1
2,3,-1,-1,1,1,0,1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
3,4,-1,-1,1,1,0,1,-1,-1,1,...,-1,-1,-1,-1,1,-1,-1,-1,-1,1
4,5,-1,1,-1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,-1


In [145]:
#Load place-voicing-manner information
phon_pvm = pd.read_csv("Resources/phon_pvm.csv")
phon_pvm.head()

,IPA_singles,Arpabet,Phoneme_ID,Place,Manner,Place_N,Manner_N,Height,Frontness,Height_N,Frontness_N
0,h,H,1,glottal,fricative,8,4,consonant,consonant,0,0
1,r,R,2,alveolar,retroflex approximant,4,6,consonant,consonant,0,0
2,w,W,3,bilabial,glide,1,8,consonant,consonant,0,0
3,j,Y,4,palatal,glide,6,8,consonant,consonant,0,0
4,b,B,5,bilabial,stop,1,1,consonant,consonant,0,0


In [146]:
#Assign pvm features for productions
prod_phon_pvm = (
    phon_pvm
    .drop(columns=[
        'IPA_singles',
        'Arpabet'
        ])
    .add_prefix("Prod_")
)
prod_phon_pvm.head()

,Prod_Phoneme_ID,Prod_Place,Prod_Manner,Prod_Place_N,Prod_Manner_N,Prod_Height,Prod_Frontness,Prod_Height_N,Prod_Frontness_N
0,1,glottal,fricative,8,4,consonant,consonant,0,0
1,2,alveolar,retroflex approximant,4,6,consonant,consonant,0,0
2,3,bilabial,glide,1,8,consonant,consonant,0,0
3,4,palatal,glide,6,8,consonant,consonant,0,0
4,5,bilabial,stop,1,1,consonant,consonant,0,0


In [147]:
#Assign pvm features for targets
target_phon_pvm = (
    phon_pvm
    .drop(columns=[
        'IPA_singles',
        'Arpabet'
        ])
    .add_prefix("Target_")
)
target_phon_pvm.head()

,Target_Phoneme_ID,Target_Place,Target_Manner,Target_Place_N,Target_Manner_N,Target_Height,Target_Frontness,Target_Height_N,Target_Frontness_N
0,1,glottal,fricative,8,4,consonant,consonant,0,0
1,2,alveolar,retroflex approximant,4,6,consonant,consonant,0,0
2,3,bilabial,glide,1,8,consonant,consonant,0,0
3,4,palatal,glide,6,8,consonant,consonant,0,0
4,5,bilabial,stop,1,1,consonant,consonant,0,0


In [148]:
#Merge all the feature information with the main dataset
df7 = (
    df6.merge(
            prod_phon_dist_features, 
            on='Prod_Phoneme_ID', 
            how='left')
        .merge(
            prod_phon_pvm, 
            on='Prod_Phoneme_ID', 
            how='left')
        .merge( 
            target_phon_dist_features, 
            on='Target_Phoneme_ID', 
            how='left')
        .merge( 
            target_phon_pvm, 
            on='Target_Phoneme_ID', 
            how='left')
        
        )
df7.columns.tolist()

['RA',
 'Project',
 'PID',
 'Test',
 'Arm',
 'Week',
 'Target',
 'Production',
 'Prod_Word_Dur',
 'NOTES',
 'Prod_Arpabet',
 'Word_ID',
 'Session_ID',
 'Prod_Word_N',
 'Prod_Phon_N',
 'PIDSESS_Code',
 'WordPhon_Code',
 'Phon_Sess_Code',
 'Word_Sess_Code',
 'Prod_Last_Phon',
 'Prod_Word_IPA',
 'Prod_Phon_IPA',
 'Prod_Phoneme_ID',
 'Target_Arpabet',
 'Target_N_Tot_Words',
 'Target_Word_IPA',
 'Target_N_Tot_Syllables',
 'Target_N_Tot_Characters',
 'Target_Phon_Arpabet',
 'Target_Phoneme_ID',
 'Target_Syll_Env',
 'Target_Word_Pos',
 'Syllable_NumID',
 'Target_Word_NumID',
 'Target_Con_Cluster',
 'Target_Clust_ID',
 'Target_Clus_Type',
 'Target_Clust_Phon_Pos',
 'Target_Clust_Phon_Env',
 'Target_Phon_IPA',
 'Prod_syllabic',
 'Prod_consonantal',
 'Prod_sonorant',
 'Prod_continuant',
 'Prod_delayed release',
 'Prod_approximant',
 'Prod_tap',
 'Prod_nasal',
 'Prod_voice',
 'Prod_spread gl',
 'Prod_constr gl',
 'Prod_labial',
 'Prod_round',
 'Prod_labiodental',
 'Prod_coronal',
 'Prod_anterior'

In [149]:
#Drop columns not needed for pvm analysis
pvm_all_data = (
    df7
    .drop(columns=[
        'RA',
        'Project',
        'Test',
        'Arm',
        'Week'
        ])
)
pvm_all_data.head()

,PID,Target,Production,Prod_Word_Dur,NOTES,Prod_Arpabet,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N,...,Target_affricate,Target_glide,Target_Place,Target_Manner,Target_Place_N,Target_Manner_N,Target_Height,Target_Frontness,Target_Height_N,Target_Frontness_N
0,15,book,B UH K,0.295646,Article (É) before word,B,1,0,1,1,...,-1,-1,bilabial,stop,1,1,consonant,consonant,0,0
1,15,book,B UH K,0.295646,Article (É) before word,UH,1,0,1,2,...,0,0,vowel,vowel,0,0,high,back,1,3
2,15,book,B UH K,0.295646,Article (É) before word,K,1,0,1,3,...,-1,-1,velar,stop,7,1,consonant,consonant,0,0
3,15,ball,B AO L,0.397365,NaN,B,2,0,2,1,...,-1,-1,bilabial,stop,1,1,consonant,consonant,0,0
4,15,ball,B AO L,0.397365,NaN,AO,2,0,2,2,...,0,0,vowel,vowel,0,0,mid-low,back,4,3


In [150]:
#Replace problem phonemes with correct ARPABET code or to a more readable format
pvm_all_data['Target_Word_IPA'] = (
    pvm_all_data['Target_Word_IPA']
    .replace(
            {
            'ɪresɚ':'əresɚ'
            },
        regex=True)
    .str.strip() #gets rid of white space at the beginning and ending of string
)

In [151]:
# Import the WAB and pauses data
pauses = pd.read_csv("Resources/pauses-v2.csv", encoding="ISO-8859-1")
pauses.head()

,Unnamed: 0,PID,Session_ID,speech_pause,start,end,dur,wab1_aq,wab1_nwf_total,wab1_ss_total,...,mean_length,total_length,index,numpause_to_speech_prop,duration_speech,durspeechtotal,duration_nonspeech,durpausetotal,pause_rate,spkrt
0,0,rPPA-05,2.0,speech,40.36,40.81,0.45,71.1,8.1,13.0,...,0.929655,26.96,19,0.965517,0.45,26.96,0.0,44.77,0.624146,1.075668
1,1,rPPA-05,2.0,speech,42.54,43.31,0.77,71.1,8.1,13.0,...,0.929655,26.96,19,0.965517,0.77,26.96,0.0,44.77,0.624146,1.075668
2,2,rPPA-05,2.0,speech,44.28,45.32,1.04,71.1,8.1,13.0,...,0.929655,26.96,19,0.965517,1.04,26.96,0.0,44.77,0.624146,1.075668
3,3,rPPA-05,2.0,speech,45.78,46.91,1.13,71.1,8.1,13.0,...,0.929655,26.96,19,0.965517,1.13,26.96,0.0,44.77,0.624146,1.075668
4,4,rPPA-05,2.0,speech,47.16,47.76,0.60,71.1,8.1,13.0,...,0.929655,26.96,19,0.965517,0.60,26.96,0.0,44.77,0.624146,1.075668


In [152]:
#change data type of session ID
pauses['Session_ID'] = pauses['Session_ID'].astype('int')

In [153]:
#see what participants are in the dataset
pauses['PID'].unique()

array(['rPPA-05', 'rPPA-04', 'rPPA-12', 'rPPA-13', 'rPPA-01', 'rPPA-16',
       'dPPA-09', 'rPPA-08', 'rPPA-15', 'rPPA-14', 'rPPA-07'],
      dtype=object)

In [154]:
# Make a code to merge things with
pauses['PIDSESS_Code'] = pauses["PID"].astype(str) + str('_') + pauses["Session_ID"].astype(str)

pauses.head()

,Unnamed: 0,PID,Session_ID,speech_pause,start,end,dur,wab1_aq,wab1_nwf_total,wab1_ss_total,...,total_length,index,numpause_to_speech_prop,duration_speech,durspeechtotal,duration_nonspeech,durpausetotal,pause_rate,spkrt,PIDSESS_Code
0,0,rPPA-05,2,speech,40.36,40.81,0.45,71.1,8.1,13.0,...,26.96,19,0.965517,0.45,26.96,0.0,44.77,0.624146,1.075668,rPPA-05_2
1,1,rPPA-05,2,speech,42.54,43.31,0.77,71.1,8.1,13.0,...,26.96,19,0.965517,0.77,26.96,0.0,44.77,0.624146,1.075668,rPPA-05_2
2,2,rPPA-05,2,speech,44.28,45.32,1.04,71.1,8.1,13.0,...,26.96,19,0.965517,1.04,26.96,0.0,44.77,0.624146,1.075668,rPPA-05_2
3,3,rPPA-05,2,speech,45.78,46.91,1.13,71.1,8.1,13.0,...,26.96,19,0.965517,1.13,26.96,0.0,44.77,0.624146,1.075668,rPPA-05_2
4,4,rPPA-05,2,speech,47.16,47.76,0.60,71.1,8.1,13.0,...,26.96,19,0.965517,0.60,26.96,0.0,44.77,0.624146,1.075668,rPPA-05_2


In [155]:
#Check column names
pauses.columns.to_list()

['Unnamed: 0',
 'PID',
 'Session_ID',
 'speech_pause',
 'start',
 'end',
 'dur',
 'wab1_aq',
 'wab1_nwf_total',
 'wab1_ss_total',
 'Session_Type',
 'Improvement_Group',
 'NWF_Improvement_Group',
 'SS_Improvement_Group',
 'numpauses_total',
 'numspeech_total',
 'mean_length',
 'total_length',
 'index',
 'numpause_to_speech_prop',
 'duration_speech',
 'durspeechtotal',
 'duration_nonspeech',
 'durpausetotal',
 'pause_rate',
 'spkrt',
 'PIDSESS_Code']

In [156]:
pauses2 = pauses[[
    'PID',
    'Session_ID',
    'wab1_aq',
    'wab1_nwf_total',
    'wab1_ss_total',
    'Session_Type',
    'numpause_to_speech_prop',
    'durspeechtotal',
    'durpausetotal',
    'pause_rate',
    'spkrt',
    'PIDSESS_Code'
]].drop_duplicates()

pauses2.reset_index(inplace=True,drop=True)
pauses2.head()

,PID,Session_ID,wab1_aq,wab1_nwf_total,wab1_ss_total,Session_Type,numpause_to_speech_prop,durspeechtotal,durpausetotal,pause_rate,spkrt,PIDSESS_Code
0,rPPA-05,2,71.1,8.1,13.0,Active,0.965517,26.96,44.77,0.624146,1.075668,rPPA-05_2
1,rPPA-05,1,76.3,8.2,15.0,Sham,1.000000,2.21,21.68,0.907493,2.262443,rPPA-05_1
2,rPPA-05,0,78.0,8.3,15.0,Baseline,1.000000,11.01,25.50,0.698439,1.180745,rPPA-05_0
3,rPPA-04,2,50.8,4.4,15.0,Active,1.000000,39.95,44.69,0.528001,0.976220,rPPA-04_2
4,rPPA-04,1,63.9,7.4,12.0,Sham,0.979167,37.13,42.62,0.534420,1.292755,rPPA-04_1


In [157]:
#Create a list of subject WAB-AQ scores at Baseline
wabaq = pauses2[pauses2['Session_Type']=='Baseline'].groupby('PID')['wab1_aq'].mean().to_frame().reset_index()
wabaq

,PID,wab1_aq
0,dPPA-09,41.7
1,rPPA-01,64.7
2,rPPA-04,73.7
3,rPPA-05,78.0
4,rPPA-07,94.0
5,rPPA-08,91.8
6,rPPA-12,73.0
7,rPPA-13,68.3
8,rPPA-14,90.0
9,rPPA-15,67.8


In [158]:
# Create bins in which to place values based on Baseline WAB-AQ scores
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

# Create labels for these bins
group_labels = ['1-10', '11-20', '21-30', '31-40','41-50','51-60','61-70','71-80','81-90','91-100']

In [159]:
# Slice the data and place it into bins
wabaq['wabaq_start']=pd.cut(wabaq["wab1_aq"], bins, labels=group_labels)
wabaq=wabaq.drop(columns=['wab1_aq'])
wabaq

,PID,wabaq_start
0,dPPA-09,41-50
1,rPPA-01,61-70
2,rPPA-04,71-80
3,rPPA-05,71-80
4,rPPA-07,91-100
5,rPPA-08,91-100
6,rPPA-12,71-80
7,rPPA-13,61-70
8,rPPA-14,81-90
9,rPPA-15,61-70


In [160]:
#Merge in starting WAB-AQ score
pauses3 = pauses2.merge(wabaq, on='PID', how='left')
pauses3['wabaq_start'] = pauses3['wabaq_start'].astype(str)
pauses3.head()

,PID,Session_ID,wab1_aq,wab1_nwf_total,wab1_ss_total,Session_Type,numpause_to_speech_prop,durspeechtotal,durpausetotal,pause_rate,spkrt,PIDSESS_Code,wabaq_start
0,rPPA-05,2,71.1,8.1,13.0,Active,0.965517,26.96,44.77,0.624146,1.075668,rPPA-05_2,71-80
1,rPPA-05,1,76.3,8.2,15.0,Sham,1.000000,2.21,21.68,0.907493,2.262443,rPPA-05_1,71-80
2,rPPA-05,0,78.0,8.3,15.0,Baseline,1.000000,11.01,25.50,0.698439,1.180745,rPPA-05_0,71-80
3,rPPA-04,2,50.8,4.4,15.0,Active,1.000000,39.95,44.69,0.528001,0.976220,rPPA-04_2,71-80
4,rPPA-04,1,63.9,7.4,12.0,Sham,0.979167,37.13,42.62,0.534420,1.292755,rPPA-04_1,71-80


In [161]:
#Drop unneeded columns
pauses4 = pauses3.drop(['PID','Session_ID'], axis=1)
pauses4.head()

,wab1_aq,wab1_nwf_total,wab1_ss_total,Session_Type,numpause_to_speech_prop,durspeechtotal,durpausetotal,pause_rate,spkrt,PIDSESS_Code,wabaq_start
0,71.1,8.1,13.0,Active,0.965517,26.96,44.77,0.624146,1.075668,rPPA-05_2,71-80
1,76.3,8.2,15.0,Sham,1.000000,2.21,21.68,0.907493,2.262443,rPPA-05_1,71-80
2,78.0,8.3,15.0,Baseline,1.000000,11.01,25.50,0.698439,1.180745,rPPA-05_0,71-80
3,50.8,4.4,15.0,Active,1.000000,39.95,44.69,0.528001,0.976220,rPPA-04_2,71-80
4,63.9,7.4,12.0,Sham,0.979167,37.13,42.62,0.534420,1.292755,rPPA-04_1,71-80


In [162]:
pauses4['PIDSESS_Code'].unique()

array(['rPPA-05_2', 'rPPA-05_1', 'rPPA-05_0', 'rPPA-04_2', 'rPPA-04_1',
       'rPPA-04_0', 'rPPA-12_1', 'rPPA-12_2', 'rPPA-12_0', 'rPPA-13_1',
       'rPPA-13_2', 'rPPA-13_0', 'rPPA-01_0', 'rPPA-01_2', 'rPPA-01_1',
       'rPPA-16_0', 'rPPA-16_1', 'rPPA-16_2', 'dPPA-09_0', 'dPPA-09_1',
       'dPPA-09_2', 'rPPA-08_1', 'rPPA-08_2', 'rPPA-08_0', 'rPPA-15_2',
       'rPPA-15_1', 'rPPA-15_0', 'rPPA-14_2', 'rPPA-14_1', 'rPPA-14_0',
       'rPPA-07_0', 'rPPA-07_1', 'rPPA-07_2'], dtype=object)

In [163]:
#merge the WAB data in with the main data
df7 = pvm_all_data.merge(pauses4, how = 'left', on = 'PIDSESS_Code').drop_duplicates()
df7.head()

,PID,Target,Production,Prod_Word_Dur,NOTES,Prod_Arpabet,Word_ID,Session_ID,Prod_Word_N,Prod_Phon_N,...,wab1_aq,wab1_nwf_total,wab1_ss_total,Session_Type,numpause_to_speech_prop,durspeechtotal,durpausetotal,pause_rate,spkrt,wabaq_start
0,15,book,B UH K,0.295646,Article (É) before word,B,1,0,1,1,...,67.8,3.8,15.0,Baseline,1.0,32.18,73.55,0.69564,1.336234,61-70
1,15,book,B UH K,0.295646,Article (É) before word,UH,1,0,1,2,...,67.8,3.8,15.0,Baseline,1.0,32.18,73.55,0.69564,1.336234,61-70
2,15,book,B UH K,0.295646,Article (É) before word,K,1,0,1,3,...,67.8,3.8,15.0,Baseline,1.0,32.18,73.55,0.69564,1.336234,61-70
3,15,ball,B AO L,0.397365,NaN,B,2,0,2,1,...,67.8,3.8,15.0,Baseline,1.0,32.18,73.55,0.69564,1.336234,61-70
4,15,ball,B AO L,0.397365,NaN,AO,2,0,2,2,...,67.8,3.8,15.0,Baseline,1.0,32.18,73.55,0.69564,1.336234,61-70


In [164]:
#Save as pvm_all_data.csv
df7.to_csv("Resources/all_data_pvm.csv", index=False, header=True)